In [10]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython import display
import numpy as np
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'
%matplotlib inline
import pygmo as pg
from pygmo import algorithm
import os
import sys
import json
from orbsim.constants import *
from orbsim.derivations import *
from orbsim.simulators import symplectic
%load_ext autoreload
%autoreload 2
import time
from random import shuffle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import inspect
import math

In [12]:
def launch_sim(psi):
    """
    launch (not really a launch since we start from LEO) a 
    single rocket with a given set of hyperparameters, return the resulting path
    """
    pos_ang, burn_ang, burnDv = psi # extract parameters from decision vector
    
    """define init params"""
    #position (where on earth do we start our burn)
    x0 = np.cos(pos_ang) * leo_radius
    y0 = np.sin(pos_ang) * leo_radius
    x0 += earth_position_X
    
    # how fast are we going when we start?
    vhat_x = - np.sin(pos_ang)
    vhat_y = np.cos(pos_ang)
    v_x = (leo_velocity/unit_velocity) * vhat_x
    v_y = (leo_velocity/unit_velocity) * vhat_y
    
    # burn vector: At what angle do we launch outward, and how hard do we push?
    burnDv_x = np.cos(burn_ang)*vhat_x - np.sin(burn_ang)*vhat_y
    burnDv_y = np.sin(burn_ang)*vhat_x + np.cos(burn_ang)*vhat_y
    
    # resultant momentum vector
    p0_x = v_x + burnDv*burnDv_y - y0
    p0_y = v_y + burnDv*burnDv_x + x0
    
    """SIMULATE"""
    print(x0,y0,p0_x,p0_y)
    result = symplectic(x0,y0,p0_x,p0_y)
    
    return result

launch_sim([20,4,100])


2663.7209379470455 5959.199638252225 -5923.754279393595 2757.395466262806


(2663.720973392498,
 5959.1997319267175,
 -5923.757036789062,
 2757.3895425085266,
 2663.716661930784,
 5959.1997319280845,
 -5923.757036792023,
 2757.3895425085266)

In [9]:
class saddle_space:
    def __init__(self):
        self.dim = 2
    
    def fitness(self,psi):
        res = launch_sim(psi)
        return res
    
    def get_bounds(self):
        return ([0,0],[99,99])
    
    def get_name(self):
        return f"saddlespace"
    
    def plot(self, w, idx):
        plt.imshow(G,vmin=-1, vmax=1, cmap='jet')
        x,y = zip(*w)
        plt.scatter(x,y,4,'k',edgecolors='face')
        plt.scatter(w[idx][0],w[idx][1],15,'y',edgecolors='k')

In [ ]:
class salimans_nes:
    def __init__(self,iter=12):
        super(salimans_nes,self).__init__()
        self.prevx,self.prevy = [],[]
        
        self.iter=iter # number of steps towards estimated gradient
    
    def evolve(self,pop):
        if len(pop) == 0:
            return pop
        sigma = 3
        alpha = 0.03 # learningrate
        
        # plotting
        plotting = False
        if plotting:
            plt.figure(figsize=(self.iter,self.iter))
            no_rows = int(self.iter/4+1)
            gs = gridspec.GridSpec(no_rows,4)
            plot_index = 0
        
        #for each iteration, jitter around starting points, and move in the
        #best direction (weighted average jitter coordinates according to 
        #fitness score)
        for i in range(self.iter):
            
            if plotting:
                ax1=plt.subplot(gs[int(i/4),plot_index])
                plot_index += 1
                if plot_index == 4:
                    plot_index = 0
                plt.imshow(G,vmin=-1, vmax=1, cmap='jet')
            
            #get the population    
            wl = pop.get_x()
            
            #do the jittering and selection
            j=0
            for w in wl:
                noise = np.random.randn(200,2)
                wp = [[min(99,max(0,x)),min(99,max(0,y))] for [x,y] in np.expand_dims(w, 0) + sigma*noise]
                
                
                
                if plotting:
                    x,y = zip(*wp)
                    plt.scatter(x,y,4,'k',edgecolors='face')
                R = np.array([prob.fitness(wi)[0] for wi in wp])
                R -= R.mean()
                R /= R.std()
                g = np.dot(R, noise)
                u = alpha * g
                w += u # mutate the population
                w = [min(99,max(0,w[0])),min(99,max(0,w[1]))] # bounds
                pop.set_x(j,w)# make the move previously selected
                j+=1
        return pop

    def get_name(self):
        return f"Oisin's big-dick omegafantastic algorithm"

In [ ]:
def pygmo_es():
    uda = salimans_nes(iter=25)  # user defined algorithm
    udp = saddle_space()  # user defined problem
    prob = pg.problem(udp) # Beautiful white snow

    archi = pg.archipelago(algo=uda, prob=udp, n=50, pop_size=30)
    archi.evolve()
    sols = archi.get_champions_f()
    idx = sols.index(min(sols))
    #print("Done!! Solutions found are: ")
    #print(archi.get_champions_f())
    udp.plot(archi.get_champions_x(),idx)

    #pop = pg.population(prob,10,3)
    #algo.evolve(pop)

In [ ]:
if __name__ == '__main__':
    pygmo_es()

In [5]:
x,y = launch_sim([21,20,20])

-3575.284610287674 5461.223413487295 -5487.360085474069 -3576.295392653341


<module 'orbsim.derivations' from 'C:\\Users\\oisin\\Work\\letomes\\code\\orbsim\\derivations.py'>